In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import joblib

In [33]:
model_path = "nurkz_bert"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
mlb = joblib.load('label_binarizer.pkl')

In [34]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [35]:
def predict_tags(text, threshold=0.5):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: torch.tensor(val).to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).cpu().numpy()[0]

    predicted_tags = [mlb.classes_[i] for i, p in enumerate(probs) if p >= threshold]
    return predicted_tags


In [37]:
sample_text = """
  Футболдан екінші лигада өнер көрсететін Оралдың "Жайығы" өз алаңында елордалық "Жейран" клубын қабылдады. Бұл биыл "Ақжайық" футбол клубының орнына құрылған аталмыш команданың өз тарихындағы алғашқы ойыны, деп хабарлайдыKazinform. Бірінші таймда 24-минутта қонақтардан Алдияр Аханов есеп ашты. Арада 2 минут өткенде, Ислам Амангелдиев таразы басын теңестірді. 33-минутта Ислам есепті 2:1-ге жеткізді. 36-минутта Магоммед Оджиев мергендік танытты. 46-минутта Темірлан Үмбетов матчтың нүктесін қойды – 4:1. Вячеслав Невольников жаттықтыратын оралдық клуб биыл бірінші лигаға жолдама алуды мақсат етіп отыр. Еске сала кетейік, бұдан бұрын Батыс Қазақстанда жаңа футбол клубы құрылғанын жазған болатынбыз. "Жайық" футбол клубы Қазақстанның екінші лигасында ойнайтын болды. """

predicted = predict_tags(sample_text, 0.2)
print("Болжамды тегтер:", predicted)


Болжамды тегтер: ['футбол']


C:\Users\dimat\AppData\Local\Temp\ipykernel_22428\577360212.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = {key: torch.tensor(val).to(device) for key, val in inputs.items()}
